In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
from utils import data_load

### Combine CMU with IMDB

In [4]:
df_imdb = data_load.load_imdb_title_basics()

In [5]:
df_imdb = df_imdb[df_imdb.title_type == "movie"]

In [6]:
df_cmu_movie = data_load.load_cmu_movie_metadata()

In [7]:
df_imdb.drop('original_title', axis='columns', inplace=True)

In [8]:
# we merge on the titles name (problem: names not unique and maybe spelling errors)
merged = df_cmu_movie.merge(df_imdb, left_on='movie_name', right_on='primary_title', how='left', suffixes=('_cmu', '_imdb'))

In [9]:
merged = merged[merged.release_date.apply(lambda r: r.year) == merged.start_year]

In [10]:
# we have duplicate entries
duplicates = merged.freebase_id.value_counts()
duplicates = duplicates[duplicates > 1]
duplicates

/m/0h1z21s    5
/m/064p159    4
/m/0bbx0_     4
/m/09vq1kn    4
/m/03cg7t2    3
             ..
/m/03cb5j0    2
/m/04g0gvz    2
/m/07scm0y    2
/m/06w9zfg    2
/m/09s60jk    2
Name: freebase_id, Length: 465, dtype: Int64

In [11]:
df_imdb_rating = data_load.load_imdb_title_ratings()

In [12]:
# keep the ones with the most imdb votes => most meaningful
df = merged[merged.freebase_id.isin(list(duplicates.index))].merge(df_imdb_rating, on='tconst', how='left')

In [13]:
dropping = []
for id in list(duplicates.index):
    dft = df[df.freebase_id == id]
    ids = list(dft.sort_values('num_votes', ascending=False).iloc[1:].tconst.values)
    dropping += ids

In [14]:
merged = merged[~merged.tconst.isin(dropping)]

In [15]:
merged.head()

,wikipedia_movie_id,freebase_id,movie_name,release_date,revenue,runtime,languages,countries,genres_cmu,tconst,title_type,primary_title,is_adult,start_year,end_year,runtime_minutes,genres_imdb
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",tt0228333,movie,Ghosts of Mars,0,2001,<NA>,98,"[Action, Horror, Sci-Fi]"
2,28463795,/m/0crgdbh,Brun bitter,1988-01-01,NaN,83.0,[Norwegian Language],[Norway],"[Crime Fiction, Drama]",tt0094806,movie,Brun bitter,0,1988,<NA>,83,"[Crime, Drama]"
4,261236,/m/01mrr1,A Woman in Flames,1983-01-01,NaN,106.0,[German Language],[Germany],[Drama],tt0083949,movie,A Woman in Flames,0,1983,<NA>,106,[Drama]
9,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,[English Language],[United States of America],"[Musical, Comedy, Black-and-white]",tt0029852,movie,Alexander's Ragtime Band,0,1938,<NA>,106,"[Drama, Music, Musical]"
10,9997961,/m/06_y2j7,Contigo y aquí,1974-01-01,NaN,0.0,[Spanish Language],[Argentina],"[Musical, Drama, Comedy]",tt0200545,movie,Contigo y aquí,0,1974,<NA>,70,"[Comedy, Drama, Musical]"


In [16]:
merged.to_pickle('../data/custom/movie_metadata.pkl')